In [1]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import cg
import scipy.sparse as sp
from time import time

# judge if A is positive definite
# https://stackoverflow.com/a/44287862/19253199
# if A is symmetric and able to be Cholesky decomposed, then A is positive definite
def is_pos_def(A):
    A=A.toarray()
    if np.array_equal(A, A.T):
        try:
            np.linalg.cholesky(A)
            print("A is positive definite")
            return True
        except np.linalg.LinAlgError:
            print("A is not positive definite")
            return False
    else:
        print("A is not positive definite")
        return False

def generate_A_b_psd(n=1000):
    A = sp.random(n, n, density=0.01, format="csr")
    A = A.T @ A
    b = np.random.rand(A.shape[0])
    print(f"Generated PSD A: {A.shape}, b: {b.shape}")
    A = sp.csr_matrix(A)
    assert is_pos_def(A)
    return A, b

def main():
    A,b = generate_A_b_psd()

    t=time()
    x_sp, exit_code = cg(A, b, atol=1e-5)
    print("scipy_cg time:", time()-t)
    t=time()
    x_my = my_cg(A, b)
    print("my_cg time:", time()-t)
    print("error:", np.linalg.norm(x_sp-x_my))


def my_cg(A, b, atol=1e-5):
    x=np.zeros(A.shape[0])
    r0=b-A@x
    p=r0.copy()
    r=r0.copy()
    k=0
    while True:
        Ap = A@p
        rTr = r.T@r
        alpha = rTr / (p.T@Ap)
        x1 = x + alpha * p
        r1 = r - alpha * Ap
        if np.linalg.norm(r1)<atol:
            break
        beta=r1.T@r1/(rTr)
        p1=r1+beta*p
        x=x1.copy()
        r=r1.copy()
        p=p1.copy()
        k+=1
    return x1

if __name__ == "__main__":
    main()



Generated PSD A: (1000, 1000), b: (1000,)
A is positive definite
scipy_cg time: 1.1941978931427002
my_cg time: 0.12070584297180176
error: 0.0003341374843124111
